In [2]:
import torch
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM
import numpy as np
import ast
import pandas as pd
import warnings
import pandas as pd

# Ignore all warnings
warnings.filterwarnings("ignore")

In [3]:
# load train set

train_set = pd.read_csv(r'E:\VOD추천 프로젝트 데이터\DATA\12.18\train_one_hot_enc_1218_1738.csv')


In [5]:
train_set.head()

,subsr,content_id,ct_cl,genre_of_ct_cl,user_preference,TimeGroup,liked,Adventure,Disaster,Martial Arts,...,Fantasy romance,Space Opera or epic sci-fi,Utopia,Dystopia,Contemporary Sci-Fi,Cyberpunk,Steampunk,Psychological thriller,Mystery,Film noir
0,65941000,id_0,영화,멜로,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,65941000,id_1,영화,액션/어드벤쳐,0,NaN,0,1,0,0,...,0,1,0,0,0,0,0,1,0,1
2,65941000,id_2,영화,애니메이션,0,NaN,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,65941000,id_3,영화,드라마,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,65941000,id_4,영화,애니메이션,0,NaN,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0


In [6]:
all_genre_list = ['Adventure', 'Disaster', 'Martial Arts', 'Military Action', 'Spy and Espionage',
                  'Superhero', 'Video game movies', 'Action comedy', 'Action crime', 'Action drama',
                  'Action-horror', 'Action thriller', 'Docudrama', 'Melodrama', 'Teen drama', 'Medical drama',
                  'Legal drama', 'Religious drama', 'Sports drama', 'Political drama', 'Anthropological drama',
                  'Philosophical drama', 'Contemporary and urban fantasy', 'Epic Fantasy', 'Fairy Tale', 'Dark Fantasy',
                  'Ghost', 'Zombie', 'Werewolf', 'Vampire', 'Monster', 'Slasher', 'Splatter and Gore', 'Body Horror',
                  'Folk Horror', 'Occult', 'Found Footage', 'Outbreak', 'Historical romance', 'Regency romance', 'Romantic drama',
                  'Romantic comedy', 'Chick Flick', 'Fantasy romance', 'Space Opera or epic sci-fi', 'Utopia', 'Dystopia', 'Contemporary Sci-Fi',
                  'Cyberpunk', 'Steampunk', 'Psychological thriller', 'Mystery', 'Film noir']

In [7]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM


if __name__ == "__main__":

    # train set 데이터 학습
    data = train_set.copy()

    sparse_features = ["subsr","content_id" ,"ct_cl", "genre_of_ct_cl", "TimeGroup"] + all_genre_list

    target = ['liked']

    label_encoders = {}

    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
        label_encoders[feat] = lbe

    # label인코더 저장
    torch.save(label_encoders,'./label_encoders_1218_2305.pth')

    fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                              for feat in sparse_features]

    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns

    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

    # train_model_input = {name: data[name][:train_size] for name in feature_names}
    # test_model_input = {name: data[name][train_size:] for name in feature_names}

    train_model_input = {name: data[name] for name in feature_names}

    device = 'cpu'
    use_cuda = True
    if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device)

model.compile("adam", "binary_crossentropy", metrics=['accuracy'])

# train model
history = model.fit(train_model_input, data[target].values, batch_size=256, epochs=1, verbose=2, validation_split=0.2)

In [ ]:
# save model
torch.save(model,'./DeepFM_epoch_1_1211.h5')